## Venn diagramm

In [ ]:
install.packages('VennDiagram')
library(VennDiagram)

In [ ]:
plot_Venn_diagramm <- function(a, b, name_a, name_b) {
    options(repr.plot.res = 100) # set a high-definition resolution for the jupyter notebooks plots (DPI)
    notebook_plot_size(2, 2)
    
    return (draw.pairwise.venn(
    length(a),
    length(b),
    sum(a %in% b),
    category = c(name_a, name_b),
    fill = c("blue", "red"),
    alpha = c(0.5, 0.5),
    cat.pos = c(0, 0),
    scaled = TRUE))
}

plot_Venn_diagramm(oncokb_genes, candl_genes, "oncokb", "candl")

In [ ]:
notebook_plot_size(6, 6)
draw.triple.venn(
length(oncokb_genes),
length(cgi_genes),
length(civic_genes),
sum(oncokb_genes %in% cgi_genes),
sum(cgi_genes %in% civic_genes),
sum(civic_genes %in% oncokb_genes),
sum(cgi_genes %in% oncokb_genes & cgi_genes %in% civic_genes),
category = c("oncokb", "cgi", "civic"),
lty = rep("blank", 3),
fill = c("light blue", "pink", "green"),
alpha = rep(0.5, 3),
cat.pos = rep(0, 3),
cat.dist = rep(0.025, 3),
scaled = TRUE)

## ggballoonplot

In [ ]:
install.packages("ggpubr")
library(ggpubr)
theme_set(theme_pubr())

impact <- read.table("../../../data/cleaned_IMPACT_mutations_180508.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE)

ggballoonplot(as.data.frame(table(impact$Hugo_Symbol,
                                  impact$Variant_Type)),
              fill = "value") + scale_fill_viridis_c(option = "C")

In [ ]:
# install.packages("ggpubr")
# library(ggpubr)
# theme_set(theme_pubr())

colnames = c("is_in_oncokb", "is_in_civic", "is_in_cgi", "is_in_candl")

r <- data.frame()

for (col in colnames) {
    r <- rbind(r, impact %>% filter(!! sym(col) == "yes") %>%
                summarise(colnames = substr(col, 7, nchar(col)),
                          oncokb = sum(is_in_oncokb == "yes"),
                          civic  = sum(is_in_civic == "yes"),
                          cgi    = sum(is_in_cgi == "yes"),
                          candl  = sum(is_in_candl == "yes")))
}
r
ggballoonplot(r[,c("oncokb", "civic", "cgi", "candl")], fill = "value") + scale_fill_viridis_c(option = "C")

In [22]:
source("../src/utils/custom_tools.R")
setup_environment("../src/utils")

In [3]:
impact <- read.table("../data/cleaned_IMPACT_mutations_180508.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE)
#impact <- add_features("../data/", impact, annotations = TRUE)

# mutation distances checker

In [ ]:
check_distance <- function(starts, ends, mut_key, result) {
    if (length(starts) <= 1)
        return (result)
    
    for (i in 2:length(starts)) {
        if (starts[[i]] <= starts[[1]] & ends[[i]] >= starts[[1]] | starts[[i]] >= starts[[1]] & starts[[i]] <= ends[[1]]) {
            r1 <- data.frame(paste(starts[[1]], ends[[1]], sep = '_'), mut_key[[1]])
            r2 <- data.frame(paste(starts[[i]], ends[[i]], sep = '_'), mut_key[[i]])
            colnames(r1) <- c("v1", "v2")
            colnames(r2) <- c("v1", "v2")
            
            
            result <- rbind(result, r1)
            result <- rbind(result, r2)
            
            starts <- starts[-1]
            starts <- starts[-i]
            
            ends <- ends[-1]
            ends <- ends[-i]
            
            mut_key <- mut_key[-1]
            mut_key <- mut_key[-i]
            
            break
        }
    }
    
    if (length(starts) <= 1)
        return (result)
    
    return (check_distance(starts[2:length(starts)], ends[2:length(starts)], mut_key[2:length(starts)], result))
}

get_pos <- function(result) {
    if (nrow(result) == 0)
        return ("")
    else
        return (toString(result$v1))
}

get_mut <- function(result) {
    if (nrow(result) == 0)
        return ("")
    else
        return (toString(result$v2))
}




dd2 <- dd %>% group_by(Tumor_Sample_Barcode, Hugo_Symbol) %>% summarise(count = n(),
                                                                       pos = get_pos(check_distance(Start_Position, End_Position, mut_key, data.frame())),
                                                                       mut_key = get_mut(check_distance(Start_Position, End_Position, mut_key, data.frame()))) %>%
                                                             filter(pos != "")
nrow(dd2)
head(dd2, 10)